In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('reggie').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

data = spark.read.csv('data/Ecommerce_Customers.csv', header=True, inferSchema=True)

In [4]:
data.printSchema()
for item in data.head(1)[0]:
    print(item)

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [5]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [6]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [7]:
assembler = VectorAssembler(
    inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'],
    outputCol='features')

In [5]:
# creates a new col named features that is a DenseVector of the inputCols
output = assembler.transform(data)

In [ ]:
# output.printSchema()
output.head(1)

In [6]:
final_data = output.select('features', 'Yearly Amount Spent')
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [7]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [ ]:
train_data.describe().show()

In [ ]:
test_data.describe().show()

In [8]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [9]:
lr_model = lr.fit(train_data)

In [10]:
test_results = lr_model.evaluate(test_data)

In [20]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 11.661596344652708|
|  11.57390703917747|
|  7.518178978326432|
|-12.141579589141827|
|0.11422045762003563|
|  3.879996735888767|
|   4.63114968523962|
| -5.024066463159443|
|   4.41450318666358|
|-1.6000234035265066|
| 18.813505687646284|
|  17.98445798938411|
| -8.153807964689292|
|-0.9035907940978518|
|-13.076389953873331|
|-17.607818503765884|
| -3.830367151685209|
|  6.415676746389181|
| -8.040113131895168|
|  4.129903095809993|
+-------------------+
only showing top 20 rows



In [13]:
test_results.rootMeanSquaredError  # true value vs predicted value

9.765382898751996

In [14]:
test_results.r2  # Model explains 98% of data in variance

0.9849184583420112

In [15]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [16]:
unlabeled_data = test_data.select('features')

In [17]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.7377203726281...|
|[30.9716756438877...|
|[31.0662181616375...|
|[31.2606468698795...|
|[31.3091926408918...|
|[31.3662121671876...|
|[31.5147378578019...|
|[31.5316044825729...|
|[31.5761319713222...|
|[31.6005122003032...|
|[31.6098395733896...|
|[31.8854062999117...|
|[31.9120759292006...|
|[31.9365486184489...|
|[31.9563005605233...|
|[31.9673209478824...|
|[31.9764800614612...|
|[32.0085045178551...|
|[32.0123007682454...|
+--------------------+
only showing top 20 rows



In [18]:
predictions = lr_model.transform(unlabeled_data)   # features to predicted yearly amomunt spent

In [19]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 396.9787547279748|
|[30.7377203726281...| 450.2068351570524|
|[30.9716756438877...| 487.1204307785663|
|[31.0662181616375...| 461.0748727968162|
|[31.2606468698795...|421.21241079933134|
|[31.3091926408918...|428.84072110404486|
|[31.3662121671876...| 425.9577328712453|
|[31.5147378578019...|494.83655445962086|
|[31.5316044825729...|  432.101102542699|
|[31.5761319713222...| 542.8266073928548|
|[31.6005122003032...|460.35934580345065|
|[31.6098395733896...|426.56109166172405|
|[31.8854062999117...| 398.2570809371648|
|[31.9120759292006...|388.43830709980557|
|[31.9365486184489...| 440.2757748492015|
|[31.9563005605233...| 564.7337502509647|
|[31.9673209478824...|449.58020839133746|
|[31.9764800614612...|  324.178769287711|
|[32.0085045178551...|451.23733416065056|
|[32.0123007682454...|488.81514997014824|
+--------------------+------------